In [0]:
# ALS is a kind of block coordinate descent
# https://www.quora.com/What-is-the-Alternating-Least-Squares-method-in-recommendation-systems-And-why-does-this-algorithm-work-intuition-behind-this

import numpy as np
np.random.seed(0)
n=50
m=200
rank = 3
u = np.ones((n,rank)) + np.random.rand(n,rank)
v = np.random.rand(m,rank)
x = u @ v.T

In [0]:
maxiter = 20
x = u @ v.T
for k in range(rank):
  u_guess = np.ones((u.shape[0],1))
  error = np.copy(x)
  for i in range(maxiter):
    error_old = np.copy(error)
    if rank==1:
      # https://en.wikipedia.org/wiki/Ordinary_least_squares
      # https://en.wikipedia.org/wiki/Vector_projection
      # https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process
      # https://en.wikipedia.org/wiki/Projection_(linear_algebra)
      # restric x to the space spanned by u_guess [u_guess @ v_guess.T: is the projection of x onto the space spanned by u_guess]
      v_guess = x.T @ u_guess / np.linalg.norm(u_guess)**2 # pinv
      u_guess = x @ v_guess / np.linalg.norm(v_guess)**2
    else:
      v_guess = (np.linalg.pinv(u_guess) @ x).T # np.linalg.inv(u.T @ u) @ u.T
      # v_guess = v_guess / np.linalg.norm(v_guess,axis=0)[None,:]
      u_guess = x @ np.linalg.pinv(v_guess.T)
    error = u_guess @ v_guess.T - x
    print(np.linalg.norm(error)/np.linalg.norm( u @ v.T)*100)
    if np.linalg.norm(error-error_old) < 1e-1:
      print('----')
      x = x - u_guess @ v_guess.T
      break
    if i==maxiter-1:
      raise Exception('maxiter should be increased')

4.684642854071546
4.684420699592938
4.6844206994376165
----
3.7139383928213086
3.576234071512057
3.303296058236832
3.011873402959579
2.8490557200539146
2.7890126863389435
2.7704508386659197
2.7650382734752235
2.763487295780315
2.7630450943051676
2.762919198939009
----
7.832876700321155e-13
1.7696041995297214e-14
----


In [0]:
# interesting random initial guess [use with rank>1, converges very fast]
x = u @ v.T
# u_guess = np.ones((u.shape[0],rank))
u_guess = np.random.rand(u.shape[0],rank)
error = np.copy(x)
for i in range(maxiter):
  error_old = np.copy(error)
  v_guess = (np.linalg.pinv(u_guess) @ x).T
  u_guess = x @ np.linalg.pinv(v_guess.T)
  error = u_guess @ v_guess.T - x
  print(np.linalg.norm(error)/np.linalg.norm( u @ v.T)*100)
  if np.linalg.norm(error-error_old) < 1e-1:
    break

1.1405504281298252e-13
1.2247142201330877e-13


**Orthogonal Matching Pursuit (OMP)**

In [0]:
# https://en.wikipedia.org/wiki/Matching_pursuit
# TODO: remove duplicate rows in dictionary ('dictionary' has to have full rank)
np.random.seed(0)
nnz = 1
atoms = 10
dictionary = np.random.rand(atoms,x.shape[1])
dictionary_normalised = dictionary / np.linalg.norm(dictionary,axis=1)[:,None]

# MP
code = np.zeros((x.shape[0],dictionary.shape[0]))
residual = np.copy(x)
for i in range(atoms+1):
  idx = np.argmax(np.linalg.norm(residual @ dictionary_normalised.T,axis=0))
  c = residual @ dictionary_normalised[idx,:].T # equivalent to residual @ pinv(...)
  residual = residual - c[:,None] @ dictionary_normalised[idx,:][None,:]
  code[:,idx] = code[:,idx] + c
  print(idx,'\t',np.linalg.norm(residual))
print(np.linalg.norm(x - code @ dictionary_normalised))

print('----------------------------------------------')

# OMP
code = np.zeros((x.shape[0],dictionary.shape[0]))
residual = np.copy(x)
idx_rng = []
for i in range(atoms+1):
  idx = np.argmax(np.linalg.norm(residual @ dictionary_normalised.T,axis=0))
  idx_rng.append(idx)
  c = residual @ np.linalg.pinv(dictionary_normalised[idx_rng,:]) # project on all chosen atoms [main difference]
  residual = residual - c @ dictionary_normalised[idx_rng,:]
  code[:,idx_rng] = code[:,idx_rng] + c
  print(idx,'\t',np.linalg.norm(residual))
print(np.linalg.norm(x - code @ dictionary_normalised))

2 	 128.50848316671798
9 	 117.64160098290526
2 	 111.64237843975843
4 	 105.42709450233501
2 	 102.02315410812881
7 	 98.43800568348956
2 	 96.32934558609661
0 	 94.42698954178785
2 	 93.36385458958539
5 	 92.16182789834791
2 	 91.45669442670032
91.45669442670032
----------------------------------------------
2 	 128.50848316671798
9 	 104.91836308801801
4 	 97.001765965746
7 	 92.83796793352802
3 	 90.90853000242043
0 	 89.68102230367364
5 	 88.59052199042182
6 	 87.7661485690893
8 	 87.55516034969796
1 	 87.5296144893351
9 	 87.5296144893351
87.5296144893351


In [0]:
# MP and OMP will give the same result if the dictionary is orthonormal || or when we include a metric of the dictionary basis vectors
dictionary_ONB = np.copy(dictionary_normalised)
u,s,dictionary_ONB = np.linalg.svd(dictionary_normalised,full_matrices=False)
residual = np.copy(x[0,:])
for idx in range(3):
    residual -= residual @ np.linalg.pinv(dictionary_ONB[idx, None, :]) @ dictionary_ONB[idx, None, :]
    print(idx, '\t', np.linalg.norm(residual))

residual = np.copy(x[0,:])
idx_rng=[0,1,2]
residual -= residual @ np.linalg.pinv(dictionary_ONB[idx_rng, :]) @ dictionary_ONB[idx_rng, :]
print('\t', np.linalg.norm(residual))


0 	 16.170325068580887
1 	 16.152216781168555
2 	 15.946493651547689
	 15.946493651547689


**Ordinary and Total "Orthogonal" least squares (OLS and TLS)**

In [0]:
# https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues/2700#2700
# https://stats.stackexchange.com/questions/13152/how-to-perform-orthogonal-regression-total-least-squares-via-pca
# Ordinary LS           [the error is orthogonal to the basis]
# TLS or Orthogonal LS  [the error is orthogonal to the prediction]

# TLS
code = np.zeros((x.shape[0],dictionary.shape[0]))
residual = np.copy(x)
idx_rng = []
for i in range(atoms+1):
  idx = np.argmax([np.linalg.norm((residual @ np.linalg.pinv(dictionary_normalised[idx, None, :]) @ dictionary_normalised[idx, None, :])) for idx in range(atoms)]) # main difference to OLS 
  # differs only when the dictionary is not orthogonal, look at the idx at the atom+1 teration [why don't we see a difference before?]
  idx_rng.append(idx)
  c = residual @ np.linalg.pinv(dictionary_normalised[idx_rng,:])
  residual = residual - c @ dictionary_normalised[idx_rng,:]
  code[:,idx_rng] = code[:,idx_rng] + c
  print(idx,'\t',np.linalg.norm(residual))
print(np.linalg.norm(x - code @ dictionary_normalised))

2 	 128.50848316671798
9 	 104.91836308801801
4 	 97.001765965746
7 	 92.83796793352802
3 	 90.90853000242043
0 	 89.68102230367364
5 	 88.59052199042182
6 	 87.7661485690893
8 	 87.55516034969796
1 	 87.5296144893351
2 	 87.5296144893351
87.5296144893351


In [0]:
idx_rng

[1, 0, 0, 0]